In [1]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize

# The SIR model differential equations.
def deriv(y, t, N, beta,gamma):
    S,I,R = y

    dSdt = -(beta*I/N)*S 
    dIdt = (beta*S/N)*I - gamma*I 
    dRdt = gamma*I 
    
    return dSdt, dIdt, dRdt

def time_evo(N,beta,gamma,I0=1,R0=0,t=np.arange(0,365)):
    # Definition of the initial conditions
    # I0 and R0 denotes the number of initial infected people (I0) 
    # and the number of people that recovered and are immunized (R0)
    
    # t ise the timegrid
    
    S0=N-I0-R0  # number of people that can still contract the virus
    
    # Initial conditions vector
    y0 = S0, I0, R0

    # Integrate the SIR equations over the time grid, t.
    ret = odeint(deriv, y0, t, args=(N,beta,gamma))
    S, I, R = np.transpose(ret)
    
    return (t,S,I,R)

vector_regions = ['nord', 'centro', 'sud', 'isole']#,'nolombardia','lombardia']
time_window    = 5
for r in range(len(vector_regions)):
    fit_region = vector_regions[r]

    if fit_region =='nord':
        region    = ['Lombardia','Veneto','Emilia-Romagna','Liguria','Piemonte','Valle d\'Aosta','P.A. Trento','P.A. Bolzano','Friuli Venezia Giulia'] 
        n_regions = len(region)
    elif fit_region =='centro':
        region    = ['Toscana','Marche','Umbria','Lazio','Abruzzo','Molise']
        n_regions = len(region)
    elif fit_region =='sud':
        region    = ['Puglia','Calabria','Basilicata','Campania']
        n_regions = len(region)
    elif fit_region =='isole':
        region    = ['Sicilia','Sardegna']
        n_regions = len(region)
    
    elif  fit_region =='italia': 
        region    = 'Italia'
        n_regions = 1
    elif fit_region =='nolombardia':
        region    = ['Abruzzo','Basilicata','P.A. Bolzano','Calabria','Campania','Emilia-Romagna','Friuli Venezia Giulia','Lazio','Liguria','Marche','Molise','Piemonte','Puglia','Sardegna','Sicilia','Toscana','P.A. Trento','Umbria','Valle d\'Aosta','Veneto']
        n_regions = len(region)    
    elif fit_region =='lombardia':
        region    = ['Lombardia']
        n_regions = 1
    print(fit_region)

    popolation_regions = np.array([  1304970,      559084,        533050,   1947131,   5801692,         4459477,                1215220,5879082, 1550640,    10060574,  1525271,  305617,    4356406, 4029053, 1639591,  4999891,  3729641,       541380,  882015,          125666, 4905854])
    name_regions       = np.array(['Abruzzo','Basilicata','P.A. Bolzano','Calabria','Campania','Emilia-Romagna','Friuli Venezia Giulia','Lazio','Liguria','Lombardia','Marche','Molise','Piemonte','Puglia','Sardegna','Sicilia','Toscana','P.A. Trento','Umbria','Valle d\'Aosta','Veneto'])
    regions            = np.vstack((name_regions,popolation_regions))

    mask_reg = []
    for i in range(n_regions):
        mask_reg.append(regions[0,:] == region[i])
    mask_reg = np.array(mask_reg)
    
    data = pd.read_csv('https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv')
    N = 0
    xxx = []
    yyy = []
    zzz = []
    for i in range(n_regions):
        N += int(regions[1,mask_reg[i]])
        mask_REG=data['denominazione_regione']==region[i]
        xxx.append(data.loc[mask_REG,'totale_casi'])
        yyy.append(data.loc[mask_REG,'deceduti'])
        zzz.append(data.loc[mask_REG,'dimessi_guariti'])

    ydata       = np.array(np.sum(xxx,axis=0))
    ydata_death = np.array(np.sum(yyy,axis=0))
    ydata_rec   = np.array(np.sum(zzz,axis=0))
    ydata_inf   = ydata-ydata_rec-ydata_death
    xdata       = pd.to_numeric(range(ydata.shape[0]))
    today       = len(xdata)
    
    def minimizer(R0,t1=today-time_window,t2=today): #7
    
        #true data
        ydata_inf_2=np.array(ydata_inf[t1:t2])
        xdata_2=np.arange(0,len(ydata_inf_2))

        #model
        fin_result=time_evo(N,0.1*R0,0.1,I0=ydata_inf_2[0])
        i_vec=fin_result[2]
        i_vec_2=i_vec[0:len(xdata_2)]

        #average error
        error=np.sum(np.abs(ydata_inf_2-i_vec_2)/ydata_inf_2)*100

        return error

    minimizer_vec=np.vectorize(minimizer)

    xgrid    = np.arange(0.1,1.3,0.01)
    ygrid    = minimizer_vec(xgrid)
    r0_ideal = round(xgrid[np.argmin(ygrid)],2)
    print('r0_ideal for the '+fit_region+': ',r0_ideal)

    ydata_inf_2 = np.array(ydata_inf[today-time_window:today])
    xdata_2     = np.arange(0,len(ydata_inf_2))
    print('ydata_inf.shape '+fit_region+': ',ydata_inf.shape)
    print('ydata_inf for the '+fit_region+': ',ydata_inf)
    print('ydata_inf_2 for the '+fit_region+': ',ydata_inf_2)

    fin_result  = time_evo(N,0.1*r0_ideal,0.1,I0=ydata_inf_2[0])

    t=fin_result[0]
    s_vec=fin_result[1]
    i_vec=fin_result[2]
    r_vec=fin_result[3]
    
    
    def minimizer_gen(t1,t2):

        xgrid=np.arange(0.1,7.2,0.01)
        ygrid=minimizer_vec(xgrid,t1=t1,t2=t2)
        r0_ideal=round(xgrid[np.argmin(ygrid)],2)

        return r0_ideal
    
    r0_time=[]
    
    for i in range(today-(time_window-1)): 
        min_val=minimizer_gen(i,i+time_window) 
        r0_time.append(min_val)
        print(i,min_val)
      
    if fit_region =='nord':
        r0_time_nord=np.array(r0_time)
    elif fit_region =='centro':
        r0_time_centro=np.array(r0_time)
    elif fit_region =='sud':
        r0_time_sud=np.array(r0_time)
    elif fit_region =='isole':
        r0_time_isole=np.array(r0_time)
    elif fit_region =='nolombardia':
        r0_time_nolombardia=np.array(r0_time)
    elif fit_region =='lombardia':
        r0_time_lombardia=np.array(r0_time)
    r0_time.clear()
    
df_r0=pd.DataFrame(pd.to_datetime(np.arange(len(r0_time_nord)),unit='D',origin='2020-02-28'))
df_r0['nord']   = r0_time_nord
df_r0['centro'] = r0_time_centro
df_r0['sud']    = r0_time_sud
df_r0['isole']  = r0_time_isole
#df_r0['nolombardia']    = r0_time_nolombardia
#df_r0['lombardia']      = r0_time_lombardia


df_r0.columns   = ['Data','nord','centro','sud','isole']#,'nolombardia','lombardia']

df_r0.to_csv('output/r0_regions.csv',index=False)

nord
r0_ideal for the nord:  0.96
ydata_inf.shape nord:  (61,)
ydata_inf for the nord:  [  219   304   379   576   797  1004  1502  1751  2114  2485  2973  3490
  4498  5608  7050  7406  9230 11127 12837 15062 17234 19254 21543 23445
 26893 30373 34200 37022 39950 42647 45066 48373 51748 54074 56732 57456
 58760 60934 62369 63856 66273 68600 69965 70435 71302 72662 73715 75310
 77015 78155 78502 79568 80301 80585 81351 81655 81430 80862 81110 80224
 79978]
ydata_inf_2 for the nord:  [81430 80862 81110 80224 79978]
0 4.23
1 3.99
2 4.44
3 3.78
4 3.44
5 3.48
6 2.7
7 2.75
8 2.7
9 2.98
10 3.12
11 3.34
12 2.8
13 2.67
14 2.5
15 2.83
16 2.63
17 2.43
18 2.35
19 2.19
20 2.11
21 2.12
22 2.15
23 2.26
24 2.07
25 1.91
26 1.74
27 1.67
28 1.65
29 1.6
30 1.61
31 1.53
32 1.35
33 1.3
34 1.24
35 1.27
36 1.3
37 1.28
38 1.3
39 1.31
40 1.2
41 1.14
42 1.13
43 1.16
44 1.19
45 1.19
46 1.2
47 1.14
48 1.11
49 1.09
50 1.09
51 1.07
52 1.04
53 1.04
54 1.0
55 0.96
56 0.96
centro
r0_ideal for the centro:  0.95
ydata_i

/Users/nestrada/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:104: RuntimeWarning: invalid value encountered in true_divide


2 0.1
3 5.27
4 4.89
5 3.14
6 3.62
7 4.2
8 3.19
9 4.29
10 3.73
11 3.31
12 3.61
13 2.66
14 2.85
15 3.14
16 3.18
17 3.0
18 3.08
19 2.82
20 2.91
21 3.01
22 2.7
23 3.03
24 2.48
25 2.2
26 1.99
27 1.87
28 1.83
29 2.0
30 1.99
31 2.02
32 1.9
33 1.75
34 1.65
35 1.54
36 1.54
37 1.54
38 1.47
39 1.32
40 1.33
41 1.26
42 1.23
43 1.25
44 1.19
45 1.19
46 1.14
47 1.12
48 1.1
49 1.08
50 1.04
51 1.05
52 1.01
53 1.06
54 1.03
55 1.01
56 1.02
isole
r0_ideal for the isole:  1.06
ydata_inf.shape isole:  (61,)
ydata_inf for the isole:  [   0    3    3    2    2    2    7    5    6   18   18   27   38   62
   71   80  118  150  169  197  254  308  341  399  525  667  779  923
 1024 1194 1348 1557 1654 1811 1912 2030 2149 2219 2324 2408 2515 2589
 2634 2680 2733 2818 2843 2889 2933 2964 2971 2951 2973 3011 3052 3066
 3064 3096 3120 3118 3124]
ydata_inf_2 for the isole:  [3064 3096 3120 3118 3124]
0 0.1
1 0.1
2 0.1
3 3.29
4 3.75
5 5.58
6 0.23
7 5.22
8 6.01
9 3.49
10 4.73
11 4.22
12 3.83
13 3.15
14 3.17
15 3.49
16 